In [1]:
import pandas
import json
from pprint import pprint

In [2]:
# Load and preprocess world data
# Preprocessing is very specific to your input data. 

# In this case, the country name and data property were 
# included in the same data column, so preprocessing involved
# separating that information into two separate columns. 

data = pandas.read_csv('worlddata.csv')
data['Country'] = data['Time']
cols = data.columns.tolist()
cols = cols[-1:] + cols[:-1]
data = data[cols]

data['Country'] = data['Country'].apply(lambda x: x[x.find('[')+1:x.find(']')])
data['Time'] = data['Time'].apply(lambda x: x[:x.find('[')]) # This label setting is based on the required data format for MapModelViz

data

,Country,Time,2014,2015,2016,2017,2018,2019,2020,2021,...,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064
0,American Samoa,total pop,5.559900e+04,5.701660e+04,5.837320e+04,5.967420e+04,6.092430e+04,6.212770e+04,6.328810e+04,6.440890e+04,...,8.963980e+04,9.013220e+04,9.061280e+04,9.108180e+04,9.153910e+04,9.198480e+04,9.241890e+04,9.284170e+04,9.325300e+04,9.365290e+04
1,Australia,total pop,2.410000e+07,2.420000e+07,2.430000e+07,2.440000e+07,2.440000e+07,2.450000e+07,2.460000e+07,2.460000e+07,...,2.390000e+07,2.390000e+07,2.380000e+07,2.370000e+07,2.370000e+07,2.360000e+07,2.350000e+07,2.340000e+07,2.340000e+07,2.330000e+07
2,Brunei Darussalam,total pop,4.231960e+05,4.284250e+05,4.333140e+05,4.378820e+05,4.421480e+05,4.461290e+05,4.498400e+05,4.532960e+05,...,4.801050e+05,4.792610e+05,4.783590e+05,4.774010e+05,4.763900e+05,4.753260e+05,4.742120e+05,4.730490e+05,4.718390e+05,4.705840e+05
3,China,total pop,1.380000e+09,1.380000e+09,1.380000e+09,1.390000e+09,1.390000e+09,1.390000e+09,1.390000e+09,1.390000e+09,...,1.240000e+09,1.230000e+09,1.220000e+09,1.220000e+09,1.210000e+09,1.210000e+09,1.200000e+09,1.190000e+09,1.190000e+09,1.180000e+09
4,Fiji,total pop,8.987600e+05,9.067920e+05,9.149910e+05,9.233050e+05,9.316900e+05,9.401100e+05,9.485330e+05,9.569340e+05,...,1.170000e+06,1.180000e+06,1.180000e+06,1.180000e+06,1.190000e+06,1.190000e+06,1.190000e+06,1.200000e+06,1.200000e+06,1.200000e+06
5,Micronesia,total pop,1.049370e+05,1.071330e+05,1.094730e+05,1.119240e+05,1.144610e+05,1.170630e+05,1.197160e+05,1.224040e+05,...,2.115270e+05,2.139780e+05,2.164190e+05,2.188520e+05,2.212760e+05,2.236920e+05,2.260990e+05,2.284970e+05,2.308870e+05,2.332690e+05
6,Guam,total pop,1.628960e+05,1.647830e+05,1.666170e+05,1.683990e+05,1.701300e+05,1.718130e+05,1.734460e+05,1.750330e+05,...,2.058350e+05,2.061650e+05,2.064660e+05,2.067380e+05,2.069840e+05,2.072010e+05,2.073920e+05,2.075560e+05,2.076940e+05,2.078060e+05
7,Hong Kong SAR China,total pop,7.350000e+06,7.340000e+06,7.330000e+06,7.320000e+06,7.300000e+06,7.280000e+06,7.250000e+06,7.230000e+06,...,5.560000e+06,5.500000e+06,5.440000e+06,5.390000e+06,5.330000e+06,5.280000e+06,5.220000e+06,5.160000e+06,5.110000e+06,5.050000e+06
8,Indonesia,total pop,2.610000e+08,2.630000e+08,2.650000e+08,2.670000e+08,2.690000e+08,2.710000e+08,2.730000e+08,2.750000e+08,...,3.400000e+08,3.420000e+08,3.440000e+08,3.450000e+08,3.470000e+08,3.490000e+08,3.510000e+08,3.520000e+08,3.540000e+08,3.560000e+08
9,Japan,total pop,1.270000e+08,1.260000e+08,1.250000e+08,1.240000e+08,1.240000e+08,1.230000e+08,1.220000e+08,1.210000e+08,...,9.260000e+07,9.190000e+07,9.110000e+07,9.040000e+07,8.970000e+07,8.890000e+07,8.820000e+07,8.750000e+07,8.680000e+07,8.610000e+07


In [3]:
# Load JSON File to be matched
mapdata = json.load(open('lowrescountries.geojson'))

# The added column name should link to the geojson file that will be used. 
# The nameColumn should be what is used to match to the csv data.
# There are cases where these two are the same. 

addedColumn = 'id'
nameColumn = 'name' 

In [4]:
# Run this code block if the addedColumn is not a member of the geojson properties dictionary

for feat in mapdata['features']: 
    feat['properties'][addedColumn] = feat[addedColumn]
    del feat[addedColumn]
    
with open('updated.geojson', 'w') as outfile:  
    json.dump(mapdata, outfile)

In [9]:
# Prepare data to be matched. 
# The matchColumn is what will be used to link to the provided GeoJson. 
# In this case, the only common data between the two sources is the Country and name fields of the two sources. 

matchColumn = 'Country' 

df = data.filter([matchColumn], axis=1)
df[addedColumn] = ''
df = df.drop_duplicates()

In [10]:
# This code will match column data from model data (matchColumn) and 
# geoJSON data (nameColumn)

def match(row):
    a = row[matchColumn]
    b = False
    for obj in mapdata['features']: 
        if obj['properties'][nameColumn].strip().lower() == a.strip().lower(): 
            b = obj['properties'][addedColumn]
    
    result = {}
    result[matchColumn] = a
    result[addedColumn] = b
    return pandas.Series(result)

df = df.apply(match, axis=1)

# Save the auto-matched data to address missing matches
# It is generally unavoidable that the match algorithm will not match all 
# entries. The remaining rows must be considered manually. 
df.to_csv('matched.csv',index=False)

In [11]:
# After manually fixing all of the missed matches, reload the data
df = pandas.read_csv('matched.csv')

In [12]:
# Merge the id and name column matches to the model data. 
# Order is important here! You should end up with the following first three columns: 
# 1) addedColumn 2) matchColumn 3) "Time"

merged = pandas.merge(data, df, on=matchColumn)

cols = merged.columns.tolist()
cols = cols[-1:] + cols[:-1]
merged = merged[cols]
merged = merged.rename(index=str, columns={"ID": addedColumn})

merged.to_csv('worlddata_merged.csv', index=False)

merged

,id,Country,Time,2014,2015,2016,2017,2018,2019,2020,...,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064
0,ASM,American Samoa,total pop,5.559900e+04,5.701660e+04,5.837320e+04,5.967420e+04,6.092430e+04,6.212770e+04,6.328810e+04,...,8.963980e+04,9.013220e+04,9.061280e+04,9.108180e+04,9.153910e+04,9.198480e+04,9.241890e+04,9.284170e+04,9.325300e+04,9.365290e+04
1,ASM,American Samoa,GDP with energy price elasticity effects,6.230000e+08,6.260000e+08,6.330000e+08,6.410000e+08,6.480000e+08,6.550000e+08,6.620000e+08,...,9.570000e+08,9.670000e+08,9.770000e+08,9.870000e+08,9.970000e+08,1.010000e+09,1.020000e+09,1.030000e+09,1.040000e+09,1.050000e+09
2,ASM,American Samoa,total dom energy demand,0.000000e+00,3.826370e-04,1.003970e-03,1.490360e-03,1.824110e-03,1.878690e-03,1.838540e-03,...,1.366830e-03,1.357030e-03,1.348640e-03,1.341080e-03,1.333840e-03,1.326740e-03,1.318310e-03,1.309220e-03,1.300620e-03,1.292390e-03
3,ASM,American Samoa,reserves plus stocks oil,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,ASM,American Samoa,reserves gas,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5,ASM,American Samoa,pc renewables consumption of total energy cons...,0.000000e+00,1.408720e+01,1.437770e+01,1.462890e+01,1.483770e+01,1.639270e+01,1.817140e+01,...,1.628530e+01,1.631720e+01,1.633590e+01,1.634830e+01,1.635990e+01,1.637300e+01,1.640570e+01,1.644960e+01,1.649040e+01,1.652930e+01
6,AUS,Australia,total pop,2.410000e+07,2.420000e+07,2.430000e+07,2.440000e+07,2.440000e+07,2.450000e+07,2.460000e+07,...,2.390000e+07,2.390000e+07,2.380000e+07,2.370000e+07,2.370000e+07,2.360000e+07,2.350000e+07,2.340000e+07,2.340000e+07,2.330000e+07
7,AUS,Australia,GDP with energy price elasticity effects,1.200000e+12,1.210000e+12,1.230000e+12,1.250000e+12,1.280000e+12,1.300000e+12,1.330000e+12,...,2.050000e+12,2.080000e+12,2.100000e+12,2.130000e+12,2.160000e+12,2.180000e+12,2.210000e+12,2.240000e+12,2.270000e+12,2.310000e+12
8,AUS,Australia,total dom energy demand,1.244510e+05,1.239770e+05,1.235180e+05,1.234790e+05,1.238880e+05,1.247120e+05,1.261340e+05,...,1.450540e+05,1.454800e+05,1.460540e+05,1.467400e+05,1.474880e+05,1.482770e+05,1.490260e+05,1.497570e+05,1.505310e+05,1.513450e+05
9,AUS,Australia,reserves plus stocks oil,6.425360e+05,6.452120e+05,6.481040e+05,6.485320e+05,6.461310e+05,6.412070e+05,6.342690e+05,...,1.250550e+05,1.171960e+05,1.098880e+05,1.030870e+05,9.675570e+04,9.085710e+04,8.535840e+04,8.022940e+04,7.544250e+04,7.097230e+04
